In [4]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [5]:
esa_adb_challenge_path = kagglehub.competition_download('esa-adb-challenge')
root = esa_adb_challenge_path

print("⇲ Dataset Loaded...")

100%|██████████| 282M/282M [00:02<00:00, 127MB/s]

Extracting files...


⇲ Dataset Loaded...


In [6]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [7]:
root = esa_adb_challenge_path
df = pd.read_parquet(f"{root}/train.parquet")
df_test = pd.read_parquet(f"{root}/test.parquet")

df_test.head()

,id,channel_1,channel_10,channel_11,channel_12,channel_13,channel_14,channel_15,channel_16,channel_17,...,telecommand_350,telecommand_351,telecommand_352,telecommand_353,telecommand_354,telecommand_36,telecommand_376,telecommand_38,telecommand_39,telecommand_40
0,14728321,0.13791,0.0,0.0,0.218915,0.270384,0.63107,0.314563,0.786995,0.265045,...,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0
1,14728322,0.13791,0.0,0.0,0.218915,0.270384,0.63107,0.314563,0.786995,0.265045,...,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0
2,14728323,0.13791,0.0,0.0,0.218915,0.270384,0.63107,0.314563,0.786995,0.265045,...,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0
3,14728324,0.13791,0.0,0.0,0.218915,0.270384,0.63107,0.314563,0.786995,0.265045,...,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0
4,14728325,0.13791,0.0,0.0,0.218915,0.270384,0.63107,0.314563,0.786995,0.265045,...,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0


In [8]:
!pip install pandas numpy torch pyro-ppl scikit-learn matplotlib pyarrow tabulate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 12.4 MB/s eta 0:00:00


In [9]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# Define paths and constants
ESA_ADB_PATH = root
OUTPUT_DIR = "PGMPROJECT"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load the training and test data
train_df = pd.read_parquet(os.path.join(ESA_ADB_PATH, "train.parquet"))
test_df = pd.read_parquet(os.path.join(ESA_ADB_PATH, "test.parquet"))

# (Optional) Split off a validation set from training data for threshold tuning & early stopping
val_fraction = 0.005   # e.g. 0.5% of data (~50k if 14M total)
val_size = int(len(train_df) * val_fraction)
val_df = train_df.iloc[-val_size:].copy()
train_df = train_df.iloc[:-val_size].copy()
print(f"Training data size --> {len(train_df)}, Validation data size: {len(val_df)}")

# Separate out labels (and ID) from features
train_labels = train_df.pop("is_anomaly").values  # ground truth labels for training data
val_labels = val_df.pop("is_anomaly").values
test_ids = test_df["id"].values  # keep test IDs for submission
# Remove ID columns from feature sets
train_ids = train_df.pop("id").values
val_ids = val_df.pop("id").values
test_ids = test_df.pop("id").values  # (store separately, drop from features)

# Feature scaling (fit on training data, apply to val and test)
numerical_cols = train_df.columns.tolist()  # remaining columns are features
scaler = StandardScaler()
train_df[numerical_cols] = scaler.fit_transform(train_df[numerical_cols])
val_df[numerical_cols] = scaler.transform(val_df[numerical_cols])
test_df[numerical_cols] = scaler.transform(test_df[numerical_cols])
print("Feature scaling complete.")

# Save scaler for future use
import joblib
joblib.dump(scaler, os.path.join(OUTPUT_DIR, "scaler.pkl"))

# Windowing: create overlapping windows from training data for sequential models
window_size = 250000
overlap = 50000
step = window_size - overlap
train_windows = []
for start_idx in tqdm(range(0, len(train_df) - window_size + 1, step), desc="Windowing train"):
    window = train_df.iloc[start_idx : start_idx + window_size]
    train_windows.append(window.reset_index(drop=True))
print(f"Created {len(train_windows)} overlapping windows from training data.")

# Save processed datasets for reference
train_df.to_parquet(os.path.join(OUTPUT_DIR, "train_scaled.parquet"))
val_df.to_parquet(os.path.join(OUTPUT_DIR, "val_scaled.parquet"))
test_df.to_parquet(os.path.join(OUTPUT_DIR, "test_scaled.parquet"))
print("Preprocessing artifacts saved (scaled data and scaler).")

Training data size --> 14654680, Validation data size: 73641
Feature scaling complete.


Windowing train: 100%|██████████| 73/73 [00:18<00:00,  4.02it/s]


Created 73 overlapping windows from training data.
Preprocessing artifacts saved (scaled data and scaler).


In [10]:
from sklearn.ensemble import IsolationForest

# Train Isolation Forest on training data (using only feature columns)
iso_model = IsolationForest(n_estimators=100, contamination='auto', random_state=42, verbose=0)
iso_model.fit(train_df.values)
print("Isolation Forest training complete.")

# Get anomaly scores for validation set
# IsolationForest.score_samples gives higher scores for normal points, so use negative for anomaly score
val_scores = -iso_model.score_samples(val_df.values)

# Tune threshold on validation to maximize corrected event-wise F0.5
def compute_event_metrics(true_labels, pred_labels):
    """Compute event-based metrics (TP_e, FP_e, FN_e, precision_e, recall_e, corrected precision, F0.5_corr)"""
    # Identify true anomaly events from true_labels
    true_events = []
    in_event = False
    for i, label in enumerate(true_labels):
        if label and not in_event:
            # start of a new true anomaly event
            true_start = i
            in_event = True
        if not label and in_event:
            # end of a true anomaly event
            true_events.append((true_start, i-1))
            in_event = False
    if in_event:  # if anomaly goes till end
        true_events.append((true_start, len(true_labels)-1))
    # Identify predicted anomaly events from pred_labels
    pred_events = []
    in_event = False
    for i, label in enumerate(pred_labels):
        if label and not in_event:
            pred_start = i
            in_event = True
        if not label and in_event:
            pred_events.append((pred_start, i-1))
            in_event = False
    if in_event:
        pred_events.append((pred_start, len(pred_labels)-1))
    # Calculate TP_e, FP_e, FN_e
    TP_e = 0
    matched_true = [False]*len(true_events)
    for (p_start, p_end) in pred_events:
        # check if predicted event overlaps any true event
        match = False
        for ti, (t_start, t_end) in enumerate(true_events):
            if not matched_true[ti] and p_start <= t_end and p_end >= t_start:
                # overlap means a true positive event
                TP_e += 1
                matched_true[ti] = True
                match = True
                break
        if not match:
            # no matching true event -> false positive event
            pass  # we'll compute FP_e separately as len(pred_events)-TP_e after loop
    FP_e = len(pred_events) - TP_e
    FN_e = len(true_events) - TP_e

    # Sample-level counts for corrected precision
    FP_t = np.sum((pred_labels == 1) & (true_labels == 0))  # false positive samples (time points)
    TN_t = np.sum((pred_labels == 0) & (true_labels == 0))  # true negative samples
    N_t = TN_t + FP_t  # total nominal samples
    # Compute precision, recall, corrected precision, and F0.5
    precision_e = TP_e / (TP_e + FP_e) if (TP_e + FP_e) > 0 else 0.0
    recall_e = TP_e / (TP_e + FN_e) if (TP_e + FN_e) > 0 else 0.0
    TNR_t = TN_t / N_t if N_t > 0 else 0.0
    prec_corr = precision_e * TNR_t
    beta = 0.5
    beta2 = beta**2
    if prec_corr + recall_e == 0:
        F_beta_corr = 0.0
    else:
        F_beta_corr = (1+beta2) * prec_corr * recall_e / (beta2 * prec_corr + recall_e)
    return {
        "TP_e": TP_e, "FP_e": FP_e, "FN_e": FN_e,
        "Precision_e": precision_e, "Recall_e": recall_e,
        "Prec_corr": prec_corr, f"F{beta}_corr": F_beta_corr
    }

# Find best threshold by trying various percentiles of validation anomaly scores
percentiles = [50, 90, 95, 98, 99, 99.5, 99.9]
best_F0_5 = -1
best_threshold = None
best_metrics = None
for perc in percentiles:
    thr = np.percentile(val_scores, perc)
    preds = (val_scores >= thr).astype(int)
    metrics = compute_event_metrics(val_labels, preds)
    F0_5 = metrics["F0.5_corr"]
    print(f"Threshold @ {perc}th percentile = {thr:.4f}, Event-F0.5 = {F0_5:.4f}")
    if F0_5 > best_F0_5:
        best_F0_5 = F0_5
        best_threshold = thr
        best_metrics = metrics
print(f"Best threshold selected --> {best_threshold:.4f}, Corresponding F0.5 = {best_F0_5:.4f}")

# Use the best threshold to make final predictions on test data
test_scores = -iso_model.score_samples(test_df.values)
test_pred_labels = (test_scores >= best_threshold).astype(int)
print(f"Test anomaly predictions: {np.sum(test_pred_labels)} outliers flagged out of {len(test_pred_labels)} samples.")

# Save model, history, metrics, and predictions
joblib.dump(iso_model, os.path.join(OUTPUT_DIR, "IsolationForest_model.joblib"))
# Isolation Forest has no epoch history; save an empty or single-epoch history for completeness
pd.DataFrame([{"epoch": 0, "note": "model trained"}]).to_csv(os.path.join(OUTPUT_DIR, "IsolationForest_history.csv"), index=False)
# Save metrics
metrics_df = pd.DataFrame([best_metrics])
metrics_df.to_csv(os.path.join(OUTPUT_DIR, "IsolationForest_metrics.csv"), index=False)
# Save predictions (for test set, including ID and predicted label)
predictions_df = pd.DataFrame({"id": test_ids, "is_anomaly": test_pred_labels})
predictions_df.to_parquet(os.path.join(OUTPUT_DIR, "IsolationForest_predictions.parquet"), index=False)

# Visualize anomaly scores vs threshold for a sample window of validation data
import matplotlib.pyplot as plt
sample_idx = slice(0, 1000)  # first 1000 points of validation for example
plt.figure(figsize=(10,4))
plt.plot(val_scores[sample_idx], label="Anomaly Score")
plt.axhline(best_threshold, color='r', linestyle='--', label=f"Threshold = {best_threshold:.2f}")
true_anom_positions = np.where(val_labels[sample_idx] == 1)[0]
plt.scatter(true_anom_positions, val_scores[sample_idx][true_anom_positions], color='orange', label="True Anomaly")
plt.title("Isolation Forest Anomaly Scores (sample)")
plt.xlabel("Time index")
plt.ylabel("Anomaly Score")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "IsolationForest_scores_plot.png"))
plt.close()

Isolation Forest training complete.
Threshold @ 50th percentile = 0.5328, Event-F0.5 = 0.0000
Threshold @ 90th percentile = 0.5482, Event-F0.5 = 0.0000
Threshold @ 95th percentile = 0.5529, Event-F0.5 = 0.0000
Threshold @ 98th percentile = 0.5584, Event-F0.5 = 0.0000
Threshold @ 99th percentile = 0.5619, Event-F0.5 = 0.0000
Threshold @ 99.5th percentile = 0.5655, Event-F0.5 = 0.0000
Threshold @ 99.9th percentile = 0.5718, Event-F0.5 = 0.0000
Best threshold selected --> 0.5328, Corresponding F0.5 = 0.0000
Test anomaly predictions: 24064 outliers flagged out of 521280 samples.


In [12]:
# Revised VAE training with normalized losses and β-KL scaling

import torch
import torch.nn as nn
import torch.optim as optim

# 1) INPUT SANITY CHECK: drop any constant columns to avoid div-by-zero
var_per_col = train_df.var(axis=0)
constant_cols = var_per_col[var_per_col == 0.0].index.tolist()
if constant_cols:
    print(f"Dropping {len(constant_cols)} constant cols:", constant_cols[:5], "…")
    train_df.drop(columns=constant_cols, inplace=True)
    val_df.drop(columns=constant_cols, inplace=True)
    test_df.drop(columns=constant_cols, inplace=True)

# Convert to tensors
X_train = torch.tensor(train_df.values, dtype=torch.float32)
X_val   = torch.tensor(val_df.values,   dtype=torch.float32)
input_dim = X_train.size(1)

# VAE definition with logvar clipping
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim=16):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc_mu = nn.Linear(64, latent_dim)
        self.fc_logvar = nn.Linear(64, latent_dim)
        self.fc_dec1 = nn.Linear(latent_dim, 64)
        self.fc_out  = nn.Linear(64, input_dim)
    def encode(self, x):
        h = torch.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = torch.clamp(self.fc_logvar(h), -10, 10)  # clip
        return mu, logvar
    def reparam(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
    def decode(self, z):
        h = torch.relu(self.fc_dec1(z))
        return self.fc_out(h)  # regression output
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparam(mu, logvar)
        recon = self.decode(z)
        return recon, mu, logvar

# Loss functions
mse_loss = nn.MSELoss(reduction='mean')  # mean over all dims
def loss_fn(x, recon, mu, logvar, beta=0.01):
    # recon loss per sample
    recon_l = mse_loss(recon, x)
    # KL divergence per latent dim, averaged
    kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_l + beta * kl, recon_l.item(), kl.item()

# Model, optimizer, scheduler
vae = VAE(input_dim=input_dim, latent_dim=16)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)
patience, best_val, epochs_no_imp = 5, float('inf'), 0
history = []

# Training loop
for epoch in range(100):
    vae.train()
    optimizer.zero_grad()
    recon, mu, logvar = vae(X_train)
    loss, r_l, kl_l = loss_fn(X_train, recon, mu, logvar, beta=0.01)
    loss.backward()
    optimizer.step()

    # Validation
    vae.eval()
    with torch.no_grad():
        recon_v, mu_v, logvar_v = vae(X_val)
        val_loss, val_r, val_kl = loss_fn(X_val, recon_v, mu_v, logvar_v, beta=0.01)
    history.append({
        "epoch": epoch,
        "train_loss": loss.item(),
        "train_recon": r_l,
        "train_kl": kl_l,
        "val_loss": val_loss.item(),
        "val_recon": val_r,
        "val_kl": val_kl
    })
    print(f"Epoch {epoch} ▶ train_loss={loss.item():.4f}, val_loss={val_loss.item():.4f}")
    scheduler.step(val_loss)
    # Early stopping
    if val_loss < best_val:
        best_val = val_loss
        epochs_no_imp = 0
        torch.save(vae.state_dict(), os.path.join(OUTPUT_DIR, "VAE_best.pth"))
    else:
        epochs_no_imp += 1
        if epochs_no_imp >= patience:
            print("Early stopping FOLLOWS patience.")
            break

# Load best weights
vae.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, "VAE_best.pth")))

# Compute anomaly scores by per-sample MSE
vae.eval()
with torch.no_grad():
    recon_val, _, _ = vae(X_val)
    val_errors = torch.mean((recon_val - X_val)**2, dim=1).numpy()
    recon_test, _, _ = vae(torch.tensor(test_df.values, dtype=torch.float32))
    test_errors = torch.mean((recon_test - torch.tensor(test_df.values, dtype=torch.float32))**2, dim=1).numpy()

# Threshold selection on val_errors
best_F0 = -1
for perc in [90,95,98,99,99.5,99.9]:
    thr = np.percentile(val_errors, perc)
    preds = (val_errors >= thr).astype(int)
    m = compute_event_metrics(val_labels, preds)
    print(f"{perc}th pct thr={thr:.4f} ▶ F0.5_corr={m['F0.5_corr']:.4f}")
    if m['F0.5_corr'] > best_F0:
        best_F0, best_thr, best_metrics = m['F0.5_corr'], thr, m

print(f"Chosen thr={best_thr:.4f}, F0.5_corr={best_F0:.4f}")

# Final test predictions
vae_pred = (test_errors >= best_thr).astype(int)
pd.DataFrame(history).to_csv(os.path.join(OUTPUT_DIR, "VAE_history.csv"), index=False)
pd.DataFrame([best_metrics]).to_csv(os.path.join(OUTPUT_DIR, "VAE_metrics.csv"), index=False)
pd.DataFrame({"id": test_ids, "is_anomaly": vae_pred}).to_parquet(os.path.join(OUTPUT_DIR, "VAE_predictions.parquet"), index=False)
print("Revised VAE training complete — NaNs eliminated!")

Epoch 0 ▶ train_loss=1.1238, val_loss=1.8701
Epoch 1 ▶ train_loss=1.1039, val_loss=1.8559
Epoch 2 ▶ train_loss=1.0858, val_loss=1.8439
Epoch 3 ▶ train_loss=1.0720, val_loss=1.8317
Epoch 4 ▶ train_loss=1.0619, val_loss=1.8205
Epoch 5 ▶ train_loss=1.0518, val_loss=1.8102
Epoch 6 ▶ train_loss=1.0439, val_loss=1.8004
Epoch 7 ▶ train_loss=1.0343, val_loss=1.7913
Epoch 8 ▶ train_loss=1.0270, val_loss=1.7821
Epoch 9 ▶ train_loss=1.0190, val_loss=1.7728
Epoch 10 ▶ train_loss=1.0113, val_loss=1.7644
Epoch 11 ▶ train_loss=1.0044, val_loss=1.7556
Epoch 12 ▶ train_loss=0.9979, val_loss=1.7467
Epoch 13 ▶ train_loss=0.9908, val_loss=1.7373
Epoch 14 ▶ train_loss=0.9821, val_loss=1.7281
Epoch 15 ▶ train_loss=0.9743, val_loss=1.7181
Epoch 16 ▶ train_loss=0.9656, val_loss=1.7080
Epoch 17 ▶ train_loss=0.9562, val_loss=1.6972
Epoch 18 ▶ train_loss=0.9460, val_loss=1.6862
Epoch 19 ▶ train_loss=0.9345, val_loss=1.6744
Epoch 20 ▶ train_loss=0.9227, val_loss=1.6618
Epoch 21 ▶ train_loss=0.9093, val_loss=1.648

In [13]:
# ------------- Revised VAE Cell -------------
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# 1) Split off ONLY the normal points for training
normal_df = train_df[train_labels == 0].reset_index(drop=True)
anom_df   = train_df[train_labels == 1].reset_index(drop=True)

# Further split normal_df into VAE train vs normal-only validation
norm_train_df, norm_val_df = train_test_split(
    normal_df, test_size=0.1, random_state=42
)

# Build combined validation set (norm_val + all anomalies) for evaluation
eval_val_df = pd.concat([norm_val_df, anom_df]).reset_index(drop=True)
eval_val_labels = np.concatenate([
    np.zeros(len(norm_val_df), dtype=int),
    np.ones(len(anom_df), dtype=int)
])

# Scale using the same scaler we already fitted:
X_vae_train = torch.tensor(scaler.transform(norm_train_df[numerical_cols]), dtype=torch.float32)
X_vae_valnorm = torch.tensor(scaler.transform(norm_val_df[numerical_cols]),   dtype=torch.float32)
X_vae_evalval = torch.tensor(scaler.transform(eval_val_df[numerical_cols]),   dtype=torch.float32)
X_vae_test    = torch.tensor(scaler.transform(test_df[numerical_cols]),        dtype=torch.float32)

# Define the same VAE with logvar-clamp and β-scaling
class VAE(nn.Module):
    def __init__(self, D, L=16, beta=0.01):
        super().__init__()
        self.beta = beta
        self.enc1 = nn.Linear(D, 64)
        self.mu   = nn.Linear(64, L)
        self.logv = nn.Linear(64, L)
        self.dec1 = nn.Linear(L, 64)
        self.out  = nn.Linear(64, D)
    def encode(self, x):
        h = torch.relu(self.enc1(x))
        mu     = self.mu(h)
        logvar = torch.clamp(self.logv(h), -10, 10)
        return mu, logvar
    def reparam(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        return mu + torch.randn_like(std)*std
    def decode(self, z):
        h = torch.relu(self.dec1(z))
        return self.out(h)
    def forward(self, x):
        mu, logv = self.encode(x)
        z        = self.reparam(mu, logv)
        recon    = self.decode(z)
        return recon, mu, logv

# Loss with mean‐reconstruction + β‐scaled KL‐mean
mse = nn.MSELoss(reduction='mean')
def loss_fn(x, recon, mu, logv, beta):
    recon_l = mse(recon, x)
    kl = -0.5 * torch.mean(1 + logv - mu.pow(2) - logv.exp())
    return recon_l + beta*kl, recon_l.item(), kl.item()

# Instantiate
D = X_vae_train.size(1)
model = VAE(D, L=16, beta=0.01)
opt   = optim.Adam(model.parameters(), lr=1e-3)
sched = optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', patience=2, factor=0.5)

# Early stopping on *normal-only* val
best_val = float('inf'); patience = 5; no_imp = 0
history = []

for ep in range(100):
    model.train()
    opt.zero_grad()
    recon, mu, logv = model(X_vae_train)
    loss, rl, kl = loss_fn(X_vae_train, recon, mu, logv, model.beta)
    loss.backward(); opt.step()

    # validate on normal-only
    model.eval()
    with torch.no_grad():
        recon_v, mu_v, logv_v = model(X_vae_valnorm)
        vloss, vrl, vkl = loss_fn(X_vae_valnorm, recon_v, mu_v, logv_v, model.beta)
    history.append({
        "epoch":ep,
        "train_loss":loss.item(), "train_recon":rl, "train_kl":kl,
        "val_loss":vloss.item(), "val_recon":vrl, "val_kl":vkl
    })
    print(f"Ep{ep}: train={loss.item():.4f}, val={vloss:.4f}")
    sched.step(vloss)
    if vloss < best_val:
        best_val = vloss; no_imp = 0
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "VAE_best.pth"))
    else:
        no_imp += 1
        if no_imp >= patience:
            print("VAE early stop")
            break

# Load best
model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, "VAE_best.pth")))

# Compute reconstruction errors
model.eval()
with torch.no_grad():
    rec_eval, _, _ = model(X_vae_evalval)
    eval_errs = torch.mean((rec_eval - X_vae_evalval)**2, dim=1).numpy()
    rec_test, _, _ = model(X_vae_test)
    test_errs = torch.mean((rec_test - X_vae_test)**2, dim=1).numpy()

# Threshold on the *normal-only* validation errors:
thr = np.percentile(eval_errs[:len(norm_val_df)], 99.9)
print("Using threshold = ", thr)

# Predictions on evaluation set:
eval_preds = (eval_errs >= thr).astype(int)
metrics = compute_event_metrics(eval_val_labels, eval_preds)
print("Eval Event-corrected F0.5 =", metrics["F0.5_corr"])

# Finally, predict on test:
test_preds = (test_errs >= thr).astype(int)

# Save artifacts
torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "VAE_model.pth"))
pd.DataFrame(history).to_csv(os.path.join(OUTPUT_DIR, "VAE_history.csv"), index=False)
pd.DataFrame([metrics]).to_csv(os.path.join(OUTPUT_DIR, "VAE_metrics.csv"), index=False)
pd.DataFrame({"id":test_ids, "is_anomaly":test_preds}).to_parquet(os.path.join(OUTPUT_DIR, "VAE_predictions.parquet"), index=False)

print("Recomputed VAE Event-F0.5:", metrics["F0.5_corr"])

Ep0: train=131016216.0000, val=741493.0000
Ep1: train=130249072.0000, val=735084.5625
Ep2: train=129507872.0000, val=729151.6250
Ep3: train=128792200.0000, val=723661.6875
Ep4: train=128100232.0000, val=718572.1250
Ep5: train=127451096.0000, val=713938.5625
Ep6: train=126826312.0000, val=710014.6875
Ep7: train=126248776.0000, val=706751.2500
Ep8: train=125701552.0000, val=704053.6250
Ep9: train=125141464.0000, val=701850.2500
Ep10: train=124567864.0000, val=700090.1250
Ep11: train=123973400.0000, val=698640.8750
Ep12: train=123377352.0000, val=697403.5625
Ep13: train=122738704.0000, val=696242.5625
Ep14: train=122068784.0000, val=695045.3125
Ep15: train=121354016.0000, val=693749.6250
Ep16: train=120595688.0000, val=692298.0625
Ep17: train=119800960.0000, val=690690.7500
Ep18: train=118958696.0000, val=688915.1250
Ep19: train=118058240.0000, val=686994.5000
Ep20: train=117107416.0000, val=684896.3750
Ep21: train=116094280.0000, val=682600.5000
Ep22: train=115015920.0000, val=680107.437